## Tổng quan

Chiến lược để tiền xử lý dữ liệu khi đã khám phá dữ liệu cụ thể như sau:
1. Data Cleaning
- Xóa cột `enrollee_id`
- Chỉnh sửa định dạng: sau khi sửa cần chuẩn hóa kiểu dữ liệu sang số
    + Cột `experience`: chuyển '<1', '>20' sang định dạng số phù hợp
    + Cột `company_size`: chuyển '10/49', '100-500', '1000-4999',... sang định dạng số phù hợp 
    + Cột `last_new_job`: chuyển '>4', 'never' sang định dạng số phù hợp
- Xử lý trường hợp `last_new_job > experience`
- Xử lý missing values: 
    + Với các cột `company_type, enrolled_university, education_level, gender, major_discipline`: Điền giá trị mới là "Unknown" vào các giá trị bị thiếu
    + Với các cột `experience, company_size, last_new_job`: điền giá trị thiếu bằng giá trị trung vị(median)
- Xử lý giá trị ngoại lai(Outliers): áp dụng IQR Capping cho cột `training_hours`

2. Feature Engineering: tạo thêm khoảng 3 đến 4 feature mới


## Import các thư viện và module

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  #setup đường dẫn

import numpy as np

import src.data_processing as dp

## Load dữ liệu

Dữ liệu sẽ được đọc từ file `aug_train.csv`. Các features được lưu vào header, toàn bộ dữ liệu sẽ được lưu vào data.

In [2]:
input_path = "../data/raw/aug_train.csv"

header = dp.read_header(input_path) 
data = dp.read_data(input_path) 
col_idx = {name: i for i, name in enumerate(header)} # Mỗi header tương ứng với 1 index để dễ truy cập  


print("Features:")
print(header)
print("\nData tương ứng:")
print(data[:5,:])

Features:
['enrollee_id' 'city' 'city_development_index' 'gender'
 'relevent_experience' 'enrolled_university' 'education_level'
 'major_discipline' 'experience' 'company_size' 'company_type'
 'last_new_job' 'training_hours' 'target']

Data tương ứng:
[['8949' 'city_103' '0.92' 'Male' 'Has relevent experience'
  'no_enrollment' 'Graduate' 'STEM' '>20' 'nan' 'nan' '1' '36' '1.0']
 ['29725' 'city_40' '0.7759999999999999' 'Male' 'No relevent experience'
  'no_enrollment' 'Graduate' 'STEM' '15' '50-99' 'Pvt Ltd' '>4' '47'
  '0.0']
 ['11561' 'city_21' '0.624' 'nan' 'No relevent experience'
  'Full time course' 'Graduate' 'STEM' '5' 'nan' 'nan' 'never' '83' '0.0']
 ['33241' 'city_115' '0.789' 'nan' 'No relevent experience' 'nan'
  'Graduate' 'Business Degree' '<1' 'nan' 'Pvt Ltd' 'never' '52' '1.0']
 ['666' 'city_162' '0.767' 'Male' 'Has relevent experience'
  'no_enrollment' 'Masters' 'STEM' '>20' '50-99' 'Funded Startup' '4' '8'
  '0.0']]


## Tiền xử lý dữ liệu

### Xóa các cột không cần thiết

In [3]:
# Xóa cột 'enrollee_id'
if 'enrollee_id' in header:
    index_to_delete = col_idx['enrollee_id']
    data = np.delete(data, index_to_delete, axis=1)

    # Cập nhật lại header và col_idx
    header = new_arr = np.delete(header, index_to_delete)
    col_idx = {name: i for i, name in enumerate(header)} 

print("Features:")
print(header)


Features:
['city' 'city_development_index' 'gender' 'relevent_experience'
 'enrolled_university' 'education_level' 'major_discipline' 'experience'
 'company_size' 'company_type' 'last_new_job' 'training_hours' 'target']


### Xử lý với mỗi cột có kiểu dữ liệu dạng số (numerical)   

Để giải quyết các giá trị bị thiếu của mỗi cột có kiểu dữ liệu dạng số, có thể thực hiện như sau:

- Đối với cột `experience`: giá trị của cột này còn chứa các giá trị chuỗi như '<1' và '>20' — trước tiên, cần chuyển đổi tất cả các giá trị này sang định dạng số phù hợp. Các giá trị thiếu sẽ được thay thế bằng giá trị trung vị.
- Đối với cột `company_size`: giá trị của cột này còn chứa các giá trị chuỗi như '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10/49', '100-500', '500-999' — trước tiên, cần chuyển đổi tất cả các giá trị này sang định dạng số phù hợp. Các giá trị thiếu sẽ được thay thế bằng giá trị trung vị.
- Đối với cột `last_new_job`: giá trị của cột này còn chứa các giá trị chuỗi như '>4' và 'never' — trước tiên, cần chuyển đổi tất cả các giá trị này sang định dạng số phù hợp. Các giá trị thiếu sẽ được thay thế bằng giá trị trung vị.

Xử lý trường hợp `last_new_job > experience`: bằng cách thay giá trị của last_new_job bằng experience

In [4]:
# Chuyển đổi các giá trị không hợp lý
numeric_transformers = [
    ('experience', dp.convert_experience),
    ('company_size', dp.convert_company_size),
    ('last_new_job', dp.convert_last_new_job)
]
for col_name, convert in numeric_transformers:
    idx = col_idx[col_name]
    
    col_float = convert(data[:, idx])
    
    median_val = np.nanmedian(col_float)
    col_float[np.isnan(col_float)] = median_val
    
    # data đang là mảng string, nên phải ép kiểu số về string trước khi lưu lại vào data
    data[:, idx] = col_float.astype(str)

# Sửa lại giá trị không hợp lý last_new_job > experience
idx_exp = col_idx['experience']
idx_job = col_idx['last_new_job']

exp_vals = data[:, idx_exp].astype(float)
job_vals = data[:, idx_job].astype(float)

error_mask = job_vals > exp_vals
job_vals[error_mask] = exp_vals[error_mask]
data[:, idx_job] = job_vals.astype(str)

### Xử lý với mỗi cột có kiểu dữ liệu dạng phân loại (categorical)

Để giải quyết các giá trị bị thiếu của mỗi cột có kiểu dữ liệu dạng phân loại, có thể thực hiện như sau:

- Đối với các cột `gender, enrolled_university, education_level, major_discipline, company_type`: các giá trị thiếu được thay thế bằng 'Unknown' 

In [5]:
categorical_cols = ['gender','enrolled_university', 'education_level', 
                    'major_discipline', 'company_type']

cat_idxs = [col_idx[name] for name in categorical_cols]

sub_data = data[:, cat_idxs]

sub_data[sub_data == 'nan'] = 'Unknown'

# Gán vào lại data
data[:, cat_idxs] = sub_data

### Xử lý giá trị ngoại lai cho cột `training_hours`

In [6]:
idx_hours = col_idx['training_hours']
data[:, idx_hours] = dp.handle_outliers_iqr(data[:, idx_hours])

### Dữ liệu sau khi xử lý missing values sẽ như thế nào?

In [7]:
for h in header:
    print(f"Cột {h}:")
    print(np.unique(data[:, col_idx[h]]))

Cột city:
['city_1' 'city_10' 'city_100' 'city_101' 'city_102' 'city_103' 'city_104'
 'city_105' 'city_106' 'city_107' 'city_109' 'city_11' 'city_111'
 'city_114' 'city_115' 'city_116' 'city_117' 'city_118' 'city_12'
 'city_120' 'city_121' 'city_123' 'city_126' 'city_127' 'city_128'
 'city_129' 'city_13' 'city_131' 'city_133' 'city_134' 'city_136'
 'city_138' 'city_139' 'city_14' 'city_140' 'city_141' 'city_142'
 'city_143' 'city_144' 'city_145' 'city_146' 'city_149' 'city_150'
 'city_152' 'city_155' 'city_157' 'city_158' 'city_159' 'city_16'
 'city_160' 'city_162' 'city_165' 'city_166' 'city_167' 'city_171'
 'city_173' 'city_175' 'city_176' 'city_179' 'city_18' 'city_180'
 'city_19' 'city_2' 'city_20' 'city_21' 'city_23' 'city_24' 'city_25'
 'city_26' 'city_27' 'city_28' 'city_30' 'city_31' 'city_33' 'city_36'
 'city_37' 'city_39' 'city_40' 'city_41' 'city_42' 'city_43' 'city_44'
 'city_45' 'city_46' 'city_48' 'city_50' 'city_53' 'city_54' 'city_55'
 'city_57' 'city_59' 'city_61' 'cit

## Feature engineering 

Tạo thêm các feature mới:
- `is_active_learner`: Đây là tín hiệu hành vi rõ ràng nhất. Một người đang đi học (Full time/Part time) thì khả năng cực cao họ sẽ tìm việc mới sau khi tốt nghiệp. Ngược lại, người no_enrollment thường đang ổn định.
- `experience_level`:  Hành vi của Junior và Senior rất khác nhau. Junior hay nhảy việc để tăng lương/học hỏi. Senior thường ngại thay đổi. Việc chia nhóm giúp Model bắt được quy luật này tốt hơn là để số lẻ.
- `is_stem_major`: Vì là tuyển dụng Data Scientist. Người có nền tảng STEM (Toán, Kỹ thuật) là đối tượng chính. Những ngành khác (Arts, Humanities) thường là chuyển ngành, hành vi tìm việc của họ sẽ khác.
- `stability_ratio`: Đây sẽ là một feature để xem một ứng viên có trung thành với công việc không, công thức sẽ là `stability_ratio = last_new_job / experience`. Nếu như ứng viên đó có stability_ratio gần bằng 1 nghĩa là rất trung thành, còn stability_ratio quá thấp nghĩa là ứng viên này hay nhảy việc.

In [8]:
# Feature: is_stem_major
raw_major = data[:, col_idx['major_discipline']]
is_stem = dp.create_is_stem(raw_major) 

# Feature: experience_level (0: Junior, 1: Mid, 2: Senior, 3: Expert)
exp_float = data[:, col_idx['experience']].astype(float)
exp_level = dp.create_experience_level(exp_float)

# Feature: is_active_learner (Đang đi học)
raw_enroll = data[:, col_idx['enrolled_university']]
is_learner = dp.create_enrollment_status(raw_enroll)
print(is_learner)
# Feature: stability_ratio
job_float = data[:, col_idx['last_new_job']].astype(float)
exp_float = data[:, col_idx['experience']].astype(float)
stability_col = dp.create_stability_ratio(job_float, exp_float)

if 'is_stem_major' and 'experience_level' and 'is_active_learner' and 'stability_col' not in header:

    new_features = np.column_stack((is_stem, exp_level, is_learner, stability_col))

    data = np.hstack((data, new_features.astype(str)))

    header = np.append(header, ['is_stem_major', 'experience_level', 'is_active_learner', 'stability_col'])
    col_idx = {name: i for i, name in enumerate(header)}

print("Giá trị của từng feature mới là")
for h in ['is_stem_major', 'experience_level', 'is_active_learner', 'stability_col']:
    print(f"Cột {h}:")
    print(np.unique(data[:, col_idx[h]]))

[0 0 1 ... 0 0 0]
Giá trị của từng feature mới là
Cột is_stem_major:
['0.0' '1.0']
Cột experience_level:
['0.0' '1.0' '2.0' '3.0']
Cột is_active_learner:
['0.0' '1.0']
Cột stability_col:
['0.0' '0.05' '0.06' '0.07' '0.08' '0.09' '0.1' '0.11' '0.12' '0.13'
 '0.14' '0.15' '0.16' '0.17' '0.18' '0.19' '0.2' '0.21' '0.22' '0.23'
 '0.24' '0.25' '0.26' '0.27' '0.28' '0.29' '0.3' '0.31' '0.33' '0.36'
 '0.38' '0.4' '0.42' '0.43' '0.44' '0.45' '0.5' '0.56' '0.57' '0.6' '0.62'
 '0.67' '0.71' '0.75' '0.8' '0.83' '1.0']


## Save dữ liệu đã xử lý

In [9]:
output_path = "../data/processed/aug_train_processed.csv"

dp.save_to_csv(data, header, output_path)

✅ Đã lưu file thành công tại: ../data/processed/aug_train_processed.csv


## Tổng kết và Kết luận

Hiện tại, tôi đã xử lý được dữ liệu từ một số chiến lược đề ra và lưu ra file, file `aug_train_processed.csv` này đang có một dataset sạch để có thể dùng để:
- Phân tích và trả lời một số câu hỏi để hiểu sâu về dữ liệu
- Kiểm định giả thuyết thống kê  

Việc xử lý dữ liệu còn cần phải triển khai thêm khi thực hiện modeling